In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import seaborn as sns

## Combining chunks of files

The code below combine all the chunks of files processed by the VADER analysis in batch

In [2]:
# Define the number of chunks you have
num_chunks = 15631  # Adjust the number of chunks accordingly

# Initialize an empty list to store the chunk DataFrames
chunk_dfs = []

# Loop through each chunk file and read it into a DataFrame

for i in range(0, num_chunks + 1):
    chunk_file_path = f"C:\\Users\\user\\OneDrive\\Desktop\\Research Project\\Chunk Folder\\processed_chunk_{i}.csv"  # Adjust the filename pattern as needed
    
    try:
        chunk_df = pd.read_csv(chunk_file_path)
        chunk_dfs.append(chunk_df)
    except:
        chunk_df = pd.read_csv(chunk_file_path, lineterminator='\n')
        chunk_dfs.append(chunk_df)
        continue

# Concatenate all chunk DataFrames into a single DataFrame
combined_df = pd.concat(chunk_dfs, ignore_index=True)

# Display the first few rows of the combined DataFrame
print(combined_df.head())

# Export the combined DataFrame
combined_df.to_csv('C:\\Users\\user\\OneDrive\\Desktop\\Research Project\\Final_ Data\\final_data.csv')

                    datetime         username  \
0  2021-02-05 20:05:53+00:00    HodlersDigest   
1  2021-02-05 20:05:52+00:00       Alexiana_m   
2  2021-02-05 20:05:51+00:00  MalikIrfanahme7   
3  2021-02-05 20:05:49+00:00       dreezmaliq   
4  2021-02-05 20:05:49+00:00     OwenModamwen   

                                                text  sentiment_vader  
0  Running Bitcoin: Passing The Torch From Hal Fi...              0.0  
1  #EndSARS Pure truth....Nigeria can never be be...             -1.0  
2  Challenge for @ImranKhanPTI , In March 2020 #w...              1.0  
3  @cenbank\nSpare me 2days to try withdraw my Bi...              0.0  
4  Bitcoin and Hype-investing is cool but let’s n...              1.0  


## Load Bitcoin historical price data

In [2]:
bc = pd.read_csv('C:\\Users\\user\\OneDrive\\Desktop\\Research Project\\Codes\\Binance_BTCUSDT_1h.csv')

In [3]:
bc['Date'] = pd.to_datetime(bc['Date'])
bc['Date'] = bc['Date'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
bc['Date'] = pd.to_datetime(bc['Date'])
bc.set_index('Date', inplace=True)

In [4]:
bc.head()

,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
Date,,,,,,,,
2023-10-19 23:00:00,BTCUSDT,28649.64,28717.82,28630.19,28713.71,613.28448,1.758298e+07,27099
2023-10-19 22:00:00,BTCUSDT,28604.09,28704.99,28598.60,28649.65,767.85518,2.200420e+07,28479
2023-10-19 21:00:00,BTCUSDT,28715.39,28739.73,28604.09,28604.09,1118.51697,3.205666e+07,42623
2023-10-19 20:00:00,BTCUSDT,28750.75,28882.07,28684.80,28715.38,1737.14711,4.997623e+07,59830
2023-10-19 19:00:00,BTCUSDT,28646.98,28765.76,28645.00,28750.75,1039.20935,2.984059e+07,49985


In [5]:
bc.describe()

,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
count,53988.000000,53988.000000,53988.000000,53988.000000,53988.000000,5.398800e+04,5.398800e+04
mean,20340.186699,20446.873059,20227.021767,20340.629958,3106.993845,6.812040e+07,6.012418e+04
std,15829.707450,15915.568313,15738.935921,15829.602006,4394.971369,1.054251e+08,9.368348e+04
min,2870.900000,2950.000000,2817.000000,2919.000000,0.000000,0.000000e+00,0.000000e+00
25%,7803.670000,7856.090000,7750.000000,7804.950000,971.276882,9.672007e+06,1.092250e+04
50%,13128.195000,13250.005000,13000.000000,13128.830000,1706.118517,2.666508e+07,2.716450e+04
75%,29433.650000,29507.120000,29375.062500,29433.655000,3351.395208,8.804427e+07,6.357675e+04
max,68635.120000,69000.000000,68451.190000,68633.690000,137207.188600,3.005634e+09,1.442097e+06


In [7]:
bc.shape

(53988, 8)

In [6]:
null_count = bc.isnull().sum().sum()

# Print the result
print("Number of null values:", null_count)

Number of null values: 0


## Check and process the sentiment data

In [7]:
final_df = pd.read_csv('C:\\Users\\user\\OneDrive\\Desktop\\Research Project\\Codes\\final_data.csv')

In [8]:
# Convert to datetime, handle errors by setting NaT
final_df['datetime'] = pd.to_datetime(final_df['datetime'], errors='coerce')

# Fill NaN values with the previous datetime
final_df['datetime'] = final_df['datetime'].fillna(final_df['datetime'].shift(1))

# Format datetime column
final_df['datetime'] = final_df['datetime'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S") if not pd.isnull(x) else "")

In [9]:
final_df['datetime'] = pd.to_datetime(final_df['datetime'])
final_df['datetime'] = final_df['datetime'].dt.round('H')
final_df.set_index('datetime', inplace=True)


In [10]:
final_df.head()

,Unnamed: 0,sentiment_vader
datetime,,
2021-01-02,0,1.0
2021-01-02,1,1.0
2021-01-02,2,0.0
2021-01-02,3,1.0
2021-01-02,4,0.0


In [11]:
final_df.drop(['Unnamed: 0'], axis=1, inplace= True)

In [12]:
final_df2 = pd.read_csv('C:\\Users\\user\\OneDrive\\Desktop\\Research Project\\Final_ Data\\combined_df2.csv')

In [13]:
# Convert to datetime, handle errors by setting NaT
final_df2['datetime'] = pd.to_datetime(final_df2['datetime'], errors='coerce')

# Fill NaN values with the previous datetime
final_df2['datetime'] = final_df2['datetime'].fillna(final_df2['datetime'].shift(1))

# Format datetime column
final_df2['datetime'] = final_df2['datetime'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S") if not pd.isnull(x) else "")

In [14]:
final_df2['datetime'] = pd.to_datetime(final_df2['datetime'])
final_df2['datetime'] = final_df2['datetime'].dt.round('H')
final_df2.set_index('datetime', inplace=True)

In [15]:
final_df2.head()

,Unnamed: 0,date,username,text,sentiment_vader
datetime,,,,,
2022-01-02,0,2022-01-01,priconoticias,0.4MOT TOKENS IN #LATOKEN airdrop and maybe m...,0.0
2022-01-02,1,2022-01-01,mmonlineonline,MARA for Bitcoin Exposure: Top Trade Q1 2022 h...,1.0
2022-01-02,2,2022-01-01,CoachKevinRose,Every Time the Bell Rings The Titano makes MON...,0.0
2022-01-02,3,2022-01-01,robert54177167,If your interested in losing money in 2022 kee...,1.0
2022-01-02,4,2022-01-01,WorldCoinIndex,Bitcoin price index https://t.co/o7UcHJUhC6 #U...,0.0


In [16]:
final_df2.drop(['Unnamed: 0', 'date', 'username', 'text'], axis=1, inplace = True)

In [17]:
final_df = final_df.append(final_df2)

C:\Users\user\AppData\Local\Temp\ipykernel_3192\3831104864.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(final_df2)


In [18]:
final_df.head()

,sentiment_vader
datetime,
2021-01-02,1.0
2021-01-02,1.0
2021-01-02,0.0
2021-01-02,1.0
2021-01-02,0.0


In [19]:
final_df.describe()

,sentiment_vader
count,2.290755e+07
mean,2.214518e-01
std,7.573359e-01
min,-1.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,1.000000e+00
max,1.000000e+00


In [20]:
min_date = final_df.index[0]
max_date = final_df.index[-1]
print(min_date)
print(max_date)

2021-01-02 00:00:00
2022-06-30 00:00:00


In [21]:
final_df.index[0]

Timestamp('2021-01-02 00:00:00')

Calculate Positive, Neutral and Negative Count, besides that, ratio  of postive to negative count is also calculated

In [22]:
final_df['sentiment_category'] = pd.cut(final_df['sentiment_vader'], bins=[-2, -0.01, 0.01, 2], labels=['negative', 'neutral', 'positive'])

# Resample the DataFrame to hourly intervals and group by sentiment category
hourly_sentiment_counts = final_df.groupby(pd.Grouper(freq='1H'))['sentiment_category'].value_counts().unstack().fillna(0)

# Create variables for positive, neutral, and negative sentiments within each hour
final_df['positive_count'] = hourly_sentiment_counts['positive']
final_df['neutral_count'] = hourly_sentiment_counts['neutral']
final_df['negative_count'] = hourly_sentiment_counts['negative']

# Drop the original sentiment category columns
hourly_sentiment_counts.drop(columns=['positive', 'neutral', 'negative'], inplace=True)

final_df['ratio_positive'] = final_df['positive_count']/final_df['negative_count']

In [23]:
final_df

,sentiment_vader,sentiment_category,positive_count,neutral_count,negative_count,ratio_positive
datetime,,,,,,
2021-01-02,1.0,positive,444,432,250,1.776000
2021-01-02,1.0,positive,444,432,250,1.776000
2021-01-02,0.0,neutral,444,432,250,1.776000
2021-01-02,1.0,positive,444,432,250,1.776000
2021-01-02,0.0,neutral,444,432,250,1.776000
...,...,...,...,...,...,...
2022-06-30,-1.0,negative,541,553,499,1.084168
2022-06-30,0.0,neutral,541,553,499,1.084168
2022-06-30,1.0,positive,541,553,499,1.084168


In [24]:
final_df.describe()

,sentiment_vader,positive_count,neutral_count,negative_count,ratio_positive
count,2.290755e+07,2.290755e+07,2.290755e+07,2.290755e+07,2.290755e+07
mean,2.214518e-01,8.593453e+02,7.961573e+02,4.204261e+02,2.217490e+00
std,7.573359e-01,4.464703e+02,6.742884e+02,2.813585e+02,6.722589e-01
min,-1.000000e+00,7.300000e+01,8.000000e+01,4.000000e+01,7.425743e-01
25%,0.000000e+00,6.020000e+02,5.300000e+02,2.680000e+02,1.778667e+00
50%,0.000000e+00,7.580000e+02,6.750000e+02,3.520000e+02,2.136585e+00
75%,1.000000e+00,9.670000e+02,8.630000e+02,4.720000e+02,2.550877e+00
max,1.000000e+00,5.406000e+03,1.648500e+04,3.606000e+03,1.011747e+01


Calculate the sentiment polorization, net sentiment, positive to total sentiment count and negative to total sentiment count

In [25]:
final_df['sentiment_polarization'] = (final_df['positive_count']-final_df['negative_count'])/(final_df['positive_count']+final_df['negative_count'])
final_df['net_sentiment'] = final_df['positive_count']-final_df['negative_count']
final_df['positive_to_total'] = final_df['positive_count']/(final_df['positive_count']+final_df['neutral_count']+final_df['negative_count'])
final_df['negative_to_total'] = final_df['negative_count']/(final_df['positive_count']+final_df['neutral_count']+final_df['negative_count'])

In [26]:
sentiment_data = final_df.resample('1H').mean()

C:\Users\user\AppData\Local\Temp\ipykernel_3192\3262709495.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sentiment_data = final_df.resample('1H').mean()


In [27]:
sentiment_data

,sentiment_vader,positive_count,neutral_count,negative_count,ratio_positive,sentiment_polarization,net_sentiment,positive_to_total,negative_to_total
datetime,,,,,,,,,
2021-01-01 00:00:00,0.344978,345.0,234.0,108.0,3.194444,0.523179,237.0,0.502183,0.157205
2021-01-01 01:00:00,0.232995,552.0,473.0,254.0,2.173228,0.369727,298.0,0.431587,0.198593
2021-01-01 02:00:00,0.226884,515.0,549.0,223.0,2.309417,0.395664,292.0,0.400155,0.173271
2021-01-01 03:00:00,0.232692,410.0,462.0,168.0,2.440476,0.418685,242.0,0.394231,0.161538
2021-01-01 04:00:00,0.308824,409.0,339.0,136.0,3.007353,0.500917,273.0,0.462670,0.153846
...,...,...,...,...,...,...,...,...,...
2022-06-30 20:00:00,0.072180,740.0,763.0,589.0,1.256367,0.113619,151.0,0.353728,0.281549
2022-06-30 21:00:00,0.068056,756.0,795.0,609.0,1.241379,0.107692,147.0,0.350000,0.281944
2022-06-30 22:00:00,0.075035,762.0,797.0,600.0,1.270000,0.118943,162.0,0.352941,0.277906


The sentiment data and bc data tables are merged with the Datetime index to generate the final_data data frame

In [28]:
final_data = pd.merge(sentiment_data, bc, left_index=True, right_index=True, how='outer')

In [29]:
final_data.to_csv('ori_data_for_pred.csv')

Plot a correlation map to check the correlation between all the variables

In [30]:
# Select only numeric columns
numeric_data = final_data.select_dtypes(include=[np.number])

# Compute the correlation of each numeric column with the 'Close' column
correlations = numeric_data.apply(lambda x: x.corr(final_data['Close']))

# Drop the 'Close' column from the result and sort the correlations
correlations.drop(["Close","High","Low","Open"]).sort_values(ascending=False)


sentiment_polarization    0.477346
sentiment_vader           0.441343
ratio_positive            0.438147
Volume USDT               0.389948
positive_to_total         0.337588
net_sentiment             0.313521
tradecount                0.233041
positive_count            0.093867
Volume BTC                0.070960
neutral_count            -0.001486
negative_count           -0.192618
negative_to_total        -0.473745
dtype: float64

Drop the variables that are not significant

In [31]:
final_data.drop(['Symbol','Open', 'High', 'Low' ], axis = 1, inplace = True)

Slice the data with time period of 2021-01-01 00:00:00 to 2022-01-01 00:00:00

In [32]:
data = final_data.loc['2021-01-01 00:00:00':'2022-06-30 00:00:00']

In [39]:
data.describe()

,sentiment_vader,positive_count,neutral_count,negative_count,ratio_positive,sentiment_polarization,net_sentiment,positive_to_total,negative_to_total,Close,Volume BTC,Volume USDT,tradecount
count,13027.000000,13027.000000,13027.000000,13027.000000,13027.000000,13027.000000,13027.000000,13027.000000,13027.000000,13027.000000,13027.000000,1.302700e+04,13027.000000
mean,0.226909,740.254548,661.837415,351.400015,2.245609,0.362978,388.854533,0.425300,0.198391,43930.400996,2751.434948,1.157007e+08,68341.146465
std,0.075473,304.782691,342.288658,182.925849,0.625699,0.113330,203.606584,0.047486,0.036492,10274.912444,2446.400513,9.478350e+07,45392.884274
min,-0.107122,73.000000,80.000000,40.000000,0.742574,-0.147727,-185.000000,0.036449,0.018798,17809.000000,0.000000,0.000000e+00,0.000000
25%,0.179100,546.500000,482.000000,242.000000,1.822042,0.291293,269.000000,0.394349,0.172675,36613.110000,1319.706480,5.688369e+07,38711.000000
50%,0.228417,674.000000,598.000000,309.000000,2.173077,0.369697,354.000000,0.423137,0.195379,43051.950000,2079.687700,9.104444e+07,56842.000000
75%,0.274927,855.500000,754.000000,405.000000,2.576857,0.440850,467.000000,0.452969,0.220470,50581.285000,3311.034429,1.438591e+08,84156.500000
max,0.667034,5406.000000,16485.000000,3606.000000,10.117470,0.820103,3133.000000,0.740194,0.416127,68633.690000,44239.811778,1.514465e+09,799206.000000


In [33]:
data

,sentiment_vader,positive_count,neutral_count,negative_count,ratio_positive,sentiment_polarization,net_sentiment,positive_to_total,negative_to_total,Close,Volume BTC,Volume USDT,tradecount
2021-01-01 00:00:00,0.344978,345.0,234.0,108.0,3.194444,0.523179,237.0,0.502183,0.157205,28995.13,2311.811445,6.676883e+07,58389.0
2021-01-01 01:00:00,0.232995,552.0,473.0,254.0,2.173228,0.369727,298.0,0.431587,0.198593,29409.99,5403.068471,1.583578e+08,103896.0
2021-01-01 02:00:00,0.226884,515.0,549.0,223.0,2.309417,0.395664,292.0,0.400155,0.173271,29194.65,2384.231560,6.984265e+07,57646.0
2021-01-01 03:00:00,0.232692,410.0,462.0,168.0,2.440476,0.418685,242.0,0.394231,0.161538,29278.40,1461.345077,4.276078e+07,42510.0
2021-01-01 04:00:00,0.308824,409.0,339.0,136.0,3.007353,0.500917,273.0,0.462670,0.153846,29220.31,2038.046803,5.961464e+07,55414.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-29 20:00:00,0.153259,764.0,737.0,463.0,1.650108,0.245314,301.0,0.389002,0.235743,20224.04,3566.068390,7.232491e+07,58966.0
2022-06-29 21:00:00,0.159506,790.0,768.0,467.0,1.691649,0.256961,323.0,0.390123,0.230617,20295.35,1443.382710,2.928279e+07,30903.0
2022-06-29 22:00:00,0.122931,645.0,610.0,437.0,1.475973,0.192237,208.0,0.381206,0.258274,20213.64,5882.179450,1.189329e+08,94026.0
2022-06-29 23:00:00,0.139256,622.0,564.0,401.0,1.551122,0.216031,221.0,0.391934,0.252678,20123.01,2047.527260,4.118746e+07,33123.0


Because of the time lagged variables, there are a few null values has been created in the data table. Since there are only a few null values, so all the null values are removed

In [34]:
null_count = data['sentiment_vader'].isnull().sum().sum()

# Print the result
print("Number of null values:", null_count)

Number of null values: 41


In [35]:
data.dropna(inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_3192\1368182302.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [36]:
len(data)

13027

The Min Max Scaler is used to normalize the data, this is important to train the machine learning models

In [37]:
# Convert the DataFrame to a numpy array
volume_btc_array = data.to_numpy()

# Normalize the data
scaler = MinMaxScaler()
normalized_data_2d = scaler.fit_transform(volume_btc_array)

# Convert the normalized data back to a DataFrame with the datetime index
normalized_data_df = pd.DataFrame(normalized_data_2d, index=data.index,columns= ['sentiment_vader', 'positive_count', 'neutral_count', 'negative_count', 
                                                                                 'ratio_positive', 'sentiment_polarization', 'net_sentiment', 'positive_to_total', 'negative_to_total',
                                                                                 'Close','Volume BTC','Volume USDT','trade_count'])

# Set the datetime index as the index of the DataFrame
normalized_data_df.index.name = 'datetime'

In [38]:
normalized_data_df

,sentiment_vader,positive_count,neutral_count,negative_count,ratio_positive,sentiment_polarization,net_sentiment,positive_to_total,negative_to_total,Close,Volume BTC,Volume USDT,trade_count
datetime,,,,,,,,,,,,,
2021-01-01 00:00:00,0.583991,0.051003,0.009387,0.019069,0.261536,0.693206,0.127185,0.661794,0.348345,0.220092,0.052256,0.044087,0.073059
2021-01-01 01:00:00,0.439338,0.089818,0.023956,0.060011,0.152605,0.534654,0.145570,0.561479,0.452509,0.228255,0.122131,0.104564,0.129999
2021-01-01 02:00:00,0.431446,0.082880,0.028589,0.051318,0.167132,0.561453,0.143761,0.516816,0.388779,0.224018,0.053893,0.046117,0.072129
2021-01-01 03:00:00,0.438948,0.063191,0.023286,0.035895,0.181112,0.585239,0.128692,0.508397,0.359251,0.225666,0.033032,0.028235,0.053190
2021-01-01 04:00:00,0.537289,0.063004,0.015788,0.026921,0.241579,0.670205,0.138035,0.605647,0.339890,0.224523,0.046068,0.039364,0.069336
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-29 20:00:00,0.336341,0.129571,0.040049,0.118620,0.096805,0.406105,0.146474,0.500967,0.546010,0.047517,0.080608,0.047756,0.073781
2022-06-29 21:00:00,0.344411,0.134446,0.041938,0.119742,0.101236,0.418140,0.153104,0.502561,0.533108,0.048920,0.032626,0.019335,0.038667
2022-06-29 22:00:00,0.297167,0.107257,0.032307,0.111329,0.078230,0.351264,0.118445,0.489889,0.602715,0.047312,0.132961,0.078531,0.117649


In [61]:
normalized_data_df.to_csv('final_sentiment_data.csv')